In [41]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from getpass import getpass

from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

import logging
import sys
import os

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.utilities import BingSearchAPIWrapper


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [47]:

from dotenv import load_dotenv

load_dotenv()

BING_SEARCH_URL=""
os.environ["BING_SUBSCRIPTION_KEY"] = BING_SUBSCRIPTION_KEY
os.environ["BING_SEARCH_URL"] = ""

In [26]:
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

In [25]:
search = BingSearchAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

ValidationError: 1 validation error for BingSearchAPIWrapper
__root__
  Did not find bing_search_url, please add an environment variable `BING_SEARCH_URL` which contains it, or pass  `bing_search_url` as a named parameter. (type=value_error)

In [28]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [17]:
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [29]:
agent_chain.run(input="hi, i am bob")





> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Bob! How can I assist you today?"
}
```

> Finished chain.


'Hello Bob! How can I assist you today?'

In [30]:
agent_chain.run(input="what's my name?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Your name is Bob."
}

> Finished chain.


'Your name is Bob.'

In [32]:
pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 784.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.8 MB/s eta 0:00:00
  Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [33]:
import yfinance as yf

def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'][0], 2)

# use the function
print(get_stock_price('AAPL'))


184.92


In [34]:
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from pydantic import BaseModel, Field

# input params
class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")


# function
class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


In [35]:
StockPriceTool = Tool(
    name='Get Stock Ticker price',
    func= get_stock_price,
    description="Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"
)

In [36]:
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
    StockPriceTool
]

In [37]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)


In [38]:
agent_chain.run(input="hi, i am bob")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello Bob! How can I assist you today?"
}

> Finished chain.


'Hello Bob! How can I assist you today?'

In [39]:
agent_chain.run(input="What is the price of Google stock")



> Entering new AgentExecutor chain...
```json
{
    "action": "Get Stock Ticker price",
    "action_input": "GOOGL"
}
```
Observation: 123.53
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The price of Google stock is 123.53"
}
```

> Finished chain.


'The price of Google stock is 123.53'

In [40]:
agent_chain.run(input="what's my name?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Your name is Bob"
}

> Finished chain.


'Your name is Bob'

<!-- put all togather  -->

Chat Messages
https://blog.langchain.dev/chat-models/

We are adding abstractions for the different types of chat messages.
HumanMessage: a message sent from the perspective of the human

AIMessage: a message sent from the perspective of the AI the human is interacting with

SystemMessage: a message setting the objectives the AI should follow

ChatMessage: a message allowing for arbitrary setting of role

In [51]:

import searchTools
import llamaTools
import stockTools
import llamaindexdb
import baseToolsHandler
import mathTools
from langchain.tools.python.tool import PythonREPLTool


tools = [
    #searchTools.SearchTool().getToolDefinition(),
    stockTools.StockTool().getToolDefinition(),
    llamaTools.LlamaTool(llamaindexdb=llamaindexdb.LLamaindexdb(),withmemory=False).getToolDefinition(),
    mathTools.MathTool().getToolDefinition(),
    PythonREPLTool()
]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)


template = """
You are a friendly, conversational banking assistant.
Talk about your  data  and answer any questions.It's ok if you don't know the answer.
Answer  {question} on the same language as question. Use search only if noone know answer .
"""
from langchain import PromptTemplate

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)
     
# system_message_prompt_content="You are a friendly, conversational banking assistant. Talk about your  data  and answer any questions.It's ok if you don't know the answer. Answer on the same language as question. Use search only if noone know answer. "

# llm = ChatOpenAI(temperature=0,prefix_messages=[SystemMessage(content=system_message_prompt_content)])


# Run LLM with PromptTemplate

# llm(new_prompt.format(concept="autoencoder"))
# llm(prompt.format(concept="regularization"))

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm
)

agent_chain = initialize_agent(tools, llm_chain, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)




INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
                    prefix_messages was transferred to model_kwargs.
                    Please confirm that prefix_messages is what you intended.
WARNING! prefix_messages is not default parameter.
                    prefix_messages was transferred to model_kwargs.
                    Please confirm that prefix_messages is what you intended.
WARNING! prefix_messages is not default parameter.
                    prefix_messages was transferred to model_kwargs.
                   

In [52]:
agent_chain.run(input="שמי דמיטרי")



> Entering new AgentExecutor chain...


TypeError: Object of type SystemMessage is not JSON serializable

In [38]:
agent_chain.run(input="איך אני לוקח משכנתא?")



> Entering new AgentExecutor chain...
{
    "action": "llamaindexdb",
    "action_input": "כיצד לקחת משכנתא"
}INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 22 tokens
> [retrieve] Total embedding token usage: 22 tokens
> [retrieve] Total embedding token usage: 22 tokens
> [retrieve] Total embedding token usage: 22 tokens
> [re

'כדי לקחת משכנתא בצורה מקצועית יש לעשות שלבי שלבים שונים כדי להגיע להחלטה הכי טובה עבורכם. השלבים הראשונים כוללים קבלת ייעוץ משכנתאות מצד יעדר בלתי תלוי, אישור עקרוני מהבנק, ובחירת המשכנתא המתאימה ביותר. בנוסף, יש לאסוף מידע על ההון הזמין ולהעריך את החובה החודשית שתחושב על פי המשכנתא שתבחרו. ייעוץ מקצועי יכול לחסוך לכם עשרות אלפי שקלים במצטבר במהלך חיי המשכנת'

In [39]:
agent_chain.run(input="מה שמי?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "שמך הוא דמיטרי, כפי שציינת בשאלתך לפני כמה זמן."
}

> Finished chain.


'שמך הוא דמיטרי, כפי שציינת בשאלתך לפני כמה זמן.'

In [25]:
agent_chain.run(input="מה שמי?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "כרגע אין לי מידע על שמך. אני פשוט מערכת חכמה שמתכוונת לעזור לך עם השאלות שלך. אם יש לך שאלות נוספות, אני כאן לעזור!"
}

> Finished chain.


'כרגע אין לי מידע על שמך. אני פשוט מערכת חכמה שמתכוונת לעזור לך עם השאלות שלך. אם יש לך שאלות נוספות, אני כאן לעזור!'

In [8]:
agent_chain.run(input="יש לי בתיק מנית גוגל . מה השער שלה?")



> Entering new AgentExecutor chain...
{
    "action": "Get Stock Ticker price",
    "action_input": "GOOGL"
}get ticker price for GOOGL

Observation: 123.53
Thought:{
    "action": "Final Answer",
    "action_input": "The current stock price for Google is 123.53"
}

> Finished chain.


'The current stock price for Google is 123.53'